# Random-to-Random

In [5]:
import random as r
import itertools as it
from typing import *

In [10]:
def get_deck(n):
    return list(range(1, n + 1))

In [96]:
class Node:
    def __init__(self, deck:list, marked:Set=set(), depth:int=0, probability:float=1):
        self.deck = deck
        self.marked = marked
        self.children = []
        self.depth = depth # number of shuffles that have been made
        self.probability = probability # probability of chosing this node from among its siblings
        
    def set_deck(self, deck:List):
        self.deck = deck
    
    def set_marked(self, marked:Set):
        self.marked = marked
        
    def iterate_children(self):
        for child in self.children:
            yield child
        return None
    
    def __str__(self, indent=0):
        return (" " * (indent * 6)) + ("\n" + (" " * (indent * 6))).join([
            "_" * (len(self.deck)*2 + 1),
            "|" + "|".join([" ", "*"][card in self.marked] for card in self.deck) + "|" + f"   depth: {self.depth}",
            "|" + "|".join(str(card) for card in self.deck) + "|" + f"    prob: {self.probability}",
            "‾" * (len(self.deck)*2 + 1),
        ])
    
    def __repr__(self):
        return str(self)
    
    def generate_children(self, shuffle:str):
        """
        make a copy of the node for each possible shuffle
        put those copies in the children of the current node
        compute and record the transition probabilities for each
        """
        
        
        if shuffle == "rtr":
            """
            - Remove a card u.a.r. [n choices]
            - Insert it u.a.r. [n choices]
            """
            n = len(self.deck)
            for position, card in enumerate(self.deck):
                for new_loc in range(n):
                    new_deck = self.deck.copy()
                    new_deck.remove(card)
                    new_deck.insert(new_loc, card)
                    new_marked = self.marked.copy()
                    new_marked.add(card)
                    node = Node(new_deck, new_marked, self.depth + 1, 1 / (n * n))
                    self.children.append(node)
            self.children = self.children[::-1] # to fix annoying ordering
                    
        if shuffle == "modified_rtr_1":
            """
            - Remove a card u.a.r (n choices)
            - Insert it u.a.r in any position except below it (or on the top if the card removes was at the bottom) [n - 1 choices]
            """
            n = len(self.deck)
            for position, card in enumerate(self.deck):
                for pre_new_loc in range(position + 2, position + 1 + n):
                    new_deck = self.deck.copy()
                    new_deck.remove(card)
                    new_loc = pre_new_loc % n
                    new_deck.insert(new_loc, card)
                    new_marked = self.marked.copy()
                    new_marked.add(card)
                    node = Node(new_deck, new_marked, self.depth + 1, 1 / (n * (n - 1)))
                    self.children.append(node)
            self.children = self.children[::-1] # to fix annoying ordering

        if shuffle == "modified_rtr_2":
            """
            """
            pass

In [97]:
class Tree:
    def __init__(self, n:int, shuffle:str):
        self.n = n
        self.top = Node(get_deck(n))
        self.shuffle = shuffle
    
    def __str__(self, max_depth:int=None):
        ret = ""
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if max_depth is not None and node.depth > max_depth:
                continue
            ret += node.__str__(node.depth) + "\n"
            for child in node.children:
                stack.append(child)
        return ret
    
    def __repr__(self):
        return str(self)
    
    def print(self, max_depth:int=None):
        print(self.__str__(max_depth))
    
    def expand(self, max_depth:int):
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if node.depth >= max_depth:
                continue
            if node.children == []:
                node.generate_children(self.shuffle)
            for child in node.children:
                stack.append(child)
        
    def get_leaves(self, depth:int, full:bool=False):
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if node.depth == depth:
                if not full or len(node.marked) == self.n:
                    yield node
            else:
                for child in node.children:
                    stack.append(child)
        return None
    
    def count_full_leaves(self, depth:int):
        
        self.expand(depth)
        
        data = {str(list(deck)):0 for deck in it.permutations(range(1, self.n + 1))}
        for node in self.get_leaves(depth, full=True):
            data[str(node.deck)] += 1
        for deck in data:
            print(deck, " ", data[deck])
        return None
        
    
    def P(self, t:int, sigma:list):
        if t < self.n:
            raise BaseException(f"t must be at least n={self.n}")
        if sorted(sigma) != get_deck(self.n):
            raise BaseException("sigma must be a permutation of n cards")
        
        self.expand(t)
        
        count = 0
        total = 0
        for node in self.get_leaves(t, full=True):
            total += 1
            if node.deck == sigma:
                count += 1
        return count / total

___

In [98]:
tree = Tree(3, "modified_rtr_1")

In [114]:
tree.count_full_leaves(3)

[1, 2, 3]   13
[1, 3, 2]   7
[2, 1, 3]   7
[2, 3, 1]   12
[3, 1, 2]   3
[3, 2, 1]   6


In [103]:
tree.print(max_depth=3)

_______
| | | |   depth: 0
|1|2|3|    prob: 1
‾‾‾‾‾‾‾
      _______
      | | |*|   depth: 1
      |2|3|1|    prob: 0.16666666666666666
      ‾‾‾‾‾‾‾
            _______
            | |*|*|   depth: 2
            |3|1|2|    prob: 0.16666666666666666
            ‾‾‾‾‾‾‾
                  _______
                  |*|*|*|   depth: 3
                  |1|2|3|    prob: 0.16666666666666666
                  ‾‾‾‾‾‾‾
                  _______
                  |*|*|*|   depth: 3
                  |3|1|2|    prob: 0.16666666666666666
                  ‾‾‾‾‾‾‾
                  _______
                  |*| |*|   depth: 3
                  |1|3|2|    prob: 0.16666666666666666
                  ‾‾‾‾‾‾‾
                  _______
                  | |*|*|   depth: 3
                  |3|1|2|    prob: 0.16666666666666666
                  ‾‾‾‾‾‾‾
                  _______
                  | |*|*|   depth: 3
                  |3|2|1|    prob: 0.16666666666666666
                  ‾‾‾‾‾‾‾
          